# Optunaの結果を可視化する
<hr>

## 初期設定

### matplotlibを使用する場合

In [ ]:
import optuna
import matplotlib.pyplot as plt

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization.matplotlib import plot_contour
from optuna.visualization.matplotlib import plot_edf
from optuna.visualization.matplotlib import plot_intermediate_values
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_parallel_coordinate
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization.matplotlib import plot_rank
from optuna.visualization.matplotlib import plot_slice
from optuna.visualization.matplotlib import plot_timeline


### plotlyを使用する場合

In [ ]:
import optuna
import plotly

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

### 結果のロード

In [ ]:
# 結果のロード

studyName = "reproduct_sample08"
studyStorage = "mysql://ishibashi@127.0.0.1/ishibashi_optuna_01"

study = optuna.load_study(
    study_name=studyName, 
    storage=studyStorage,
)

<hr>

## 可視化

### plot_optimization_history
- 学習履歴
- 赤がそれまでに見つけている最適解の値
- 学習回数の目安になるかも

In [ ]:
fig = plot_optimization_history(study)

# plt使うなら
# plt.show()
fig

### plot_intermediate_values
- 学習曲線を可視化するらしい
- あんまり使わないかも


In [ ]:
fig = plot_intermediate_values(study)

# plt使うなら
# plt.show()
fig

### plot_parallel_coordinate
 - 各パラメータと出力値との対応を見る
 - trialが多いとなんも見えん
 - 下みたいな感じでパラメータを絞れる

    `plot_parallel_coordinate(study, params=["lr", "n_layers"])`

In [ ]:
fig = plot_parallel_coordinate(study)

# plt使うなら
# plt.show()
fig

### plot_contour
 - 各パラメータの等高線を見ることができる
 - 見た目は一番直感的かもしれない
 - # **バチクソに重いので注意**

In [ ]:
# fig = plot_contour(study)
fig = plot_contour(study, params=["leaking_rate", "tikhonov_beta"])

# plt使うなら
# plt.show()
fig

### plot_slice
- 各パラメータを独立して見ることができる
- 曲線的な奴が出るのでわかりやすい

In [ ]:
fig = plot_slice(study)

# plt使うなら
# plt.show()
fig

### plot_param_importances
- 各パラメータの出力への貢献度を示す
- どのパラメータが一番効いているのかを分析できる

In [ ]:
fig = plot_param_importances(study)

# 時間かかったパラメータも見ることができる
# fig = plot_param_importances(study, target=lambda t: t.duration.total_seconds(), target_name="duration")

# plt使うなら
# plt.show()
fig

### plot_edf
- Empirical Sistribution Function
    - 経験的累積貢献度？とかなんとか
- これも最適化の進み具合を見る指標かな
- 使い方も読み方もわからない

In [ ]:
fig = plot_edf(study)

# plt使うなら
# plt.show()
fig

### plot_rank
- `plot_contour`に似たやつ
- こっちは等高線は出さずに，サンプルの色だけで表す

In [ ]:
fig = plot_rank(study)
# fig = plot_rank(study, params=["leaking_rate", "tikhonov_beta"])

# plt使うなら
# plt.show()
fig

### plot_timeline
- 経過時間を示す
- 表示は開始時刻と終了時刻なので，実際にどのくらい時間かかったかは自分で計算しないといけない
    - 多分表示オプションとかで制御できるが

In [ ]:
fig = plot_timeline(study)

# plt使うなら
# plt.show()
fig

<hr>

## 最適解出力

### 関数の用意
- 画像を出力するなら先にこれ実行

In [ ]:
import cupy as cp
import matplotlib.pyplot as plt

# 出力画像生成
def makeFig3(flag, title, output, GT, stde, model, rmse, nrmse):
    '''
    param flag: 画像を保存するかを管理するフラグ trueで保存
    param title: 画像タイトル
    param output: 出力データ
    param GT: 目標データ
    param stde: 目標データの標準誤差
    param model: 作ったモデルのインスタンス
    param rmse: RMSE
    param nrmse: NRMSE
    '''

    # サイズ調整
    fig = plt.figure(figsize=[5, 3])

    ax1 = fig.add_subplot(1, 1, 1)
    ax1.set_title(title, fontsize=12)
    ax1.plot(cp.asnumpy(output), color="r", linewidth=2.0, label="model output")
    ax1.grid(linestyle=":")
    ax1.set_xlabel("frame")

    ax1.fill_between(cp.asnumpy(cp.linspace(0, len(output)-1, len(output))), 
        cp.asnumpy(GT) + stde, cp.asnumpy(GT) - stde, 
        alpha=0.15, color='k', label="data std error")
    ax1.plot(cp.asnumpy(GT), color="k", label="data mean", linewidth=0.5)

    ax1.set_xlim(-10, 600)
    ax1.set_ylim(0.5, 2.0)

    ax1.legend()

    plt.show()
    plt.close()

### 最適解出力
- 値を出力
- 画像も出力

In [ ]:
best_params = study.best_params

found_nodeNum_01 = best_params["nodeNum_01"]
found_nodeNum_02 = best_params["nodeNum_02"]
found_nodeNum_03 = best_params["nodeNum_03"]
found_nodeNum_04 = best_params["nodeNum_04"]

found_leaking_rate_01 = best_params["leaking_rate_01"]
found_leaking_rate_02 = best_params["leaking_rate_02"]
found_leaking_rate_03 = best_params["leaking_rate_03"]
found_leaking_rate_04 = best_params["leaking_rate_04"]

found_intensity = best_params["intensity"]

found_tikhonov_beta = best_params["tikhonov_beta"]

best_value = study.best_value
# print(f"Found nodeNum: {found_nodeNum}, Found leaking rate:{found_leaking_rate}, Found tikhonov beta : {found_tikhonov_beta}, value : {best_value}")
print("====Found Paramators====")
print(f"nodeNum_01: {found_nodeNum_01}, leaking_rate_01: {found_leaking_rate_01}")
print(f"nodeNum_02: {found_nodeNum_02}, leaking_rate_02: {found_leaking_rate_02}")
print(f"nodeNum_03: {found_nodeNum_03}, leaking_rate_03: {found_leaking_rate_03}")
print(f"nodeNum_04: {found_nodeNum_04}, leaking_rate_04: {found_leaking_rate_04}")
print(f"intensity: {found_intensity}")
print(f"tikhonov_beta: {found_tikhonov_beta}")
print("========")
print(f"best value: {best_value}")

